In [8]:
from scrape_courses import get_courses_from_dgscene, get_readable_course_name
from scrape_events import get_all_sanctioned_events
from scrape_ratings import get_round_ratings_for_tournament
from logger import logger
import json

In [ ]:
# TODO
# generate list of all courses
# write to json
courses = get_courses_from_dgscene()
course_names = {
    course: get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

In [ ]:
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)
course_events = {}
for i, course in enumerate(course_names):
    logger.info(f'Fetching event {i}/{len(course_names)}')
    course_events[course] = get_all_sanctioned_events(course)
with open('data/course_events.json', 'w') as f:
    json.dump(course_events, f, indent=4)

In [ ]:
with open('data/course_events.json') as f:
    course_events: dict = json.load(f)
course_ratings = {}
for course in course_events:
    events = course_events[course]['events']
    course_ratings[course] = {
        event: get_round_ratings_for_tournament(event) for event in events
    }
with open('data/course_ratings.json', 'w') as f:
    json.dump(course_ratings, f, indent=4)